**To use the examples in this chapter, first run the code below to import the right libraries.**

In [ ]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random
from dataclasses import dataclass
import math

# 10 Production lines

A production line contains machines and/or persons that perform a sequence of tasks, where each machine or person is responsible for a single task. The term *server* is used for a machine or a person that performs a task.
Usually the execution of a task takes time, e.g. a drilling process, a welding process, the set-up of a machine. Timing is an important aspect of a model, as it allows answering questions about time, often performance questions ('how many products
can I make in this situation?'). 

In this chapter we discuss how to model a production line with various types of servers.



## 10.1 A simple production line
The first case is a small production line with a deterministic server (its task takes a fixed amount of time), while the
second case uses stochastic arrivals (the moment of arrival of new items varies), and a stochastic server instead (the duration of the task varies each time).

In both cases, the question is what the flow time of an item is (the amount of time that a single item is in the system),
and what the throughput of the entire system is (the number of items the production line can manufacture per time unit).

### 10.1.1 A deterministic system
The model of a deterministic system consists of a deterministic generator, a deterministic server, and an exit process.
The production line is depicted in Figure 10.1.


| Figure 10.1:  Generator `G`, server `S`, and exit `E`. |
|-|
<img src="figures/10-1.png" width=75%>
<a id='fig:10-1'></a>

Generator process `G` sends items, with constant inter-arrival time `ta`, via channel `a`, to server process `S`.
The server processes items with constant processing time `ts`, and sends items, via channel `b`, to exit process `E`.

An item contains a real value, denoting the creation time of the item, for calculating the throughput of the system and flow
time (or sojourn time) of an item in the system.
The generator process creates an item (and sets its creation time), the exit process `E` writes the measurements (the
moment in time when the item arrives in the exit process, and its creation time) to the output.
From these measurements, throughput and flow time can be calculated.

Model `M` describes the system:

In [ ]:
def M(ta, ts, N):
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    G = Generator(env, a, ta)
    S = Server(env, a, b, ts)
    E = Exit(env, b, N)
    env.run(until=E)

Parameter `ta` denotes the inter-arrival time, and is used in generator `G`.
Parameter `ts` denotes the server processing time, and is used in server `S`.
Parameter `N` denotes the number of items that must flow through the system to get a good measurement.

Generator process definition `Generator` has two parameters, channel `c_out`, and inter-arrival time `ta`.
The description of process `G` is given by:

In [ ]:
@process
def Generator(env, c_out, ta):
    while True:
        x = env.now
        yield env.execute(c_out.send(x))
        yield env.timeout(ta)

Process `G` sends an item, with the current time, and delays for `ta`, before sending the next item to server process `S`.

Server process definition `Server` has three parameters, receiving channel `c_in`, sending channel `c_out`, and server processing time `ts`:

In [ ]:
@process
def Server(env, c_in, c_out, ts):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(ts)
        yield env.execute(c_out.send(x))

The process receives an item from process `G`, processes the item during `ts` time units, and sends the item to exit process `E`. 

Exit process definition `Exit` has two parameters, receiving channel `c_in` and the length of the experiment `N`.

In [ ]:
@process
def Exit(env, c_in, N):
    for i in range(N):
        x = yield env.execute(c_in.receive())
        print(f"The Exit process received an item at t = {env.now:.1f} with flow time {env.now-x:.1f}")

The process writes current time `env.now` and item flow time `env.now - x` to the screen for each received item.
Analysis of the measurements will show that the system throughput equals $1/\mathtt{ta}$, and that the item flow time
equals `ts` (if `ta` $\ge$ `ts`).

In [ ]:
# Run the model
M(3, 1, 10) 

### 10.1.2 A stochastic system
In the next model, the generator produces items with an exponential inter-arrival time, and the server processes items
with an exponential server processing time. To compensate for the variations in time of the generator and the server, a
buffer process has been added. The model is depicted in Figure 10.2.

| Figure 10.2:  Generator `G`, Buffer `B`, server `S`, and exit `E`. |
|-|
<img src="figures/10-2.png" width=500>
<a id='fig:10-2'></a>

The model for the stochastic system runs the additional buffer process:

In [ ]:
def StochasticSystemModel(ta, ts, N):
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, ta)
    B = Buffer(env, a, b)
    S = Server(env, b, c, ts)
    E = Exit(env, c, N)
    env.run(until=E)

Generator `G` has two parameters, channel variable `a`, and variable `ta`, denoting the mean inter-arrival time.
An `exponential` distribution is used for deciding the inter-arrival time of new items, which we sample from using `u = lambda: random.exponential(ta)`. The process sends a new item to the buffer, and then is delayed for a sample of `delay = u()` time units until the next arrival.

In [ ]:
@process
def Generator(env, c_out, ta):
    u = lambda: random.exponential(ta)
    while True:
        x = env.now
        yield env.execute(c_out.send(x))
        delay = u()
        yield env.timeout(delay)

Buffer process `B` is a fifo buffer with infinite capacity, as described in chapter 9.

In [ ]:
@process
def Buffer(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]

Server `S` has three parameters, channel variables `a` and `b`, for receiving and sending items, and a variable
for the average processing time `ts`. An `exponential` distribution is used for deciding the processing time.
The process receives an item from process `B`, processes the item with the sampled processing time, and sends the item to exit process `E`.

In [ ]:
@process
def Server(env, c_in, c_out, ts):
    u = lambda: random.exponential(ts)
    while True:
        x = yield env.execute(c_in.receive())
        delay = u()
        yield env.timeout(delay)
        yield env.execute(c_out.send(x))

Exit process `E` is the same as in the previous case. In this case the throughput of the system also equals $1 / \mathtt{ta}$, and the *mean flow* can be obtained by doing an experiment and analysis of the resulting measurements (for `ta` $>$ `ts`).

In [ ]:
# Run the model
StochasticSystemModel(3, 1, 10)

## 10.2 Parallel and serial processing

In this section two different types of systems are shown: a serial and a parallel system. In a serial system the servers are positioned after each other, in a parallel system the servers are operating in parallel.
Both systems use a stochastic generator, and stochastic servers.

### 10.2.1 Serial system

The next model describes a *serial* system, where an item is processed by one server, followed by another server.
The generator and the servers are decoupled by buffers.
The model is depicted in Figure 10.3.

| Figure 10.3: A generator, two buffers, two servers, and an exit. |
|-|
<img src="figures/10-3.png" width=800>
<a id='fig:10-3'></a>


The model can be described by:

In [ ]:
def SerialSystemModel(ta, ts, N):
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    d = Channel(env)
    e = Channel(env)
    G = Generator(env, a, ta)
    B1 = Buffer(env, a, b)
    S1 = Server(env, b, c, ts)
    B2 = Buffer(env, c, d)
    S2 = Server(env, d, e, ts)
    E = Exit(env, e, N)
    env.run(until=E)

The various processes are equal to those described in the example of the stochastic system model.

In [ ]:
# Run the model
SerialSystemModel(3, 1, 10)

### 10.2.2 Parallel system

In a parallel system the servers are operating in parallel. Having several servers in parallel is useful for enlarging
the processing capacity of the task being done, or for reducing the effect of break downs of servers (when a server
breaks down, the other server continues with the task for other items).
Figure 10.4 depicts the system.

| Figure 10.4:  A model with two parallel servers. |
|-|
<img src="figures/10-4.png" width=500>
<a id='fig:10-4'></a>

Generator process `G` sends items via `a` to buffer process `B`, and process `B` sends the items in a
first-in first-out manner to the servers `S1` and `S2`. Both servers send the processed items to the exit process
`E` via channel `c`. The inter-arrival time and the two process times are assumed to be stochastic, and exponentially distributed. Items can pass each other, due to differences in processing time between the two servers.

If a server is free, and the buffer is not empty, an item is sent to a server.
If both servers are free, one server will get the item, but which one cannot be determined beforehand. (How long a
server has been idle is not taken into account.)
The model is described by:


In [ ]:
def ParallelSystemModel(ta, ts, N):
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a, ta)
    B = Buffer(env, a, b)
    S1 = Server(env, b, c, ts)
    S2 = Server(env, b, c, ts)
    E = Exit(env, c, N)
    env.run(until=E)

In [ ]:
# Run the model
ParallelSystemModel(3, 1, 10)

### 10.2.3 Parallel system with requests
To control which server gets the next item, each server must have its own channel from the buffer. In addition, the
buffer has to know when the server can receive a new item.
The latter is done with a 'request' channel, denoting that a server is free and needs a new item.
The server sends its own identity as request, the requests are administrated in the buffer.
The model is depicted in Figure 10.5.

| Figure 10.5:  A model with two parallel requesting servers. |
|-|
<img src="figures/10-5.png" width=500>
<a id='fig:10-5'></a>

In this model, the servers 'pull' an item through the line.
The model is shown below. In this model, *list comprehension* is used for the initialization and running of the two servers.
Via channel `r` an integer value, `0` or `1`, is sent to the buffer.


In [ ]:
def RequestingParallelSystemModel(ta, ts, N):
    env = Environment()
    a = Channel(env)
    b = [Channel(env) for i in range(2)]
    c = Channel(env)
    r = Channel(env)
    G = Generator(env, a, ta)
    B = BufferRequesting(env, a, b, r)
    Ss = [ServerRequesting(env, b[j], c, r, ts, j) for j in range(2)]
    E = Exit(env, c, N)
    env.run(until=E)

The items received from generator `G` are stored in the buffer in list `xs`, the requests received from the servers are stored in list `ys`.
The items and requests are removed form their respective lists in a first-in first-out manner. Process `B` is defined below.



If, there is an item present, *and*  there is a server demanding for an item, the process sends the first item to the longest waiting server.
The longest waiting server is denoted by variable `ys[0]`.
The head of the item list is denoted by `xs[0]`.
Assume the value of `ys[0]` equals `1`, then the expression `c_out[ys[0]].send(xs[0])` equals `c_out[1].send(xs[0])`, indicates that the first item of list `xs`, equals `xs[0]`, is sent to server `1`.

In [ ]:
@process
def BufferRequesting(env, c_in, c_out, c_r):
    xs = []
    ys = []
    while True:
        sending = c_out[ys[0]].send(xs[0]) if (len(xs)>0 and len(ys)>0) else None
        receiving = c_in.receive()
        request = c_r.receive()
        z = yield env.select(receiving, request, sending)
        if selected(receiving):
            xs = xs + [z]
        if selected(request):
            ys = ys + [z]
        if selected(sending):
            xs = xs[1:]
            ys = ys[1:]

The server first sends a request via channel `r` to the buffer, and waits for an item.
The item is processed, and sent to exit process `E`.

In [ ]:
@process
def ServerRequesting(env, c_in, c_out, c_r, ts, k):
    u = lambda: random.exponential(ts)
    while True:
        yield env.execute(c_r.send(k))
        x = yield env.execute(c_in.receive())
        delay = u()
        yield env.timeout(delay)
        yield env.execute(c_out.send(x))

Again the exit process `E` remains unchanged. So we can now run the model to see the results.

In [ ]:
# Run the model
RequestingParallelSystemModel(3, 1, 10)

### 10.3 Assembly

In assembly systems, components are assembled into bigger components.
These bigger components are assembled into even bigger components.
In this way, products are built, e.g. tables, chairs, computers, or cars.
In this section some simple assembly processes are described.
These systems illustrate how assembling can be performed: in industry these assembly processes are often more complicated.

An assembly work station for two components is shown in Figure 10.6.

| Figure 10.6:  Assembly for two components. |
|-|
<img src="figures/10-6.png" width=500>
<a id='fig:10-6'></a>

We first create the model for a system with assembly. Both buffers are preceded by a generator. The server is succeeded by a single exit. The model is as follows:

In [ ]:
def Assemble2PartsSystemModel(ta, ts, N):
    env = Environment()
    a = [Channel(env) for j in range(2)]
    c = [Channel(env) for j in range(2)]
    b = Channel(env)
    Gs = [Generator(env, a[j], ta) for j in range(2)]
    Bs = [Buffer(env, a[j], c[j]) for j in range(2)]
    S = ServerAssembly(env, c, b)
    E = Exit(env, b, N)
    env.run(until=E)

The assembly process server `S` is preceded by buffers. The server receives an item from each buffer `B`, before starting
assembly. The received items are assembled into one new item, a list of its (sub-)items.
The description of the assembly server is shown below.

The process takes a list of channels `c` (which corresponds to `c_in` in the process definition below) to receive items from the preceding buffers.
The output channel `b` (which corresponds to `c_out` in the process definition below) is used to send the assembled component away to the next process.

First, the assembly process receives an item from both buffers. All buffers are queried at the same time, since it is unknown which buffer has components available. If the first buffer reacts first, and sends an item, it is received with channel `c[0]` and stored in `v[0]` in the first alternative. The next step is then to receive the second component from the second buffer, and store it (`v[1] = yield env.execute(c_in[1].receive())`). The second alternative does the same, but with the channels and stored items swapped. When both components have been received, the assembled product is sent away.

In [ ]:
@process
def ServerAssembly(env, c_in, c_out):
    v = [None, None]
    while True:
        receive_part1 = c_in[0].receive()
        receive_part2 = c_in[1].receive()
        x = yield env.select(receive_part1, receive_part2) 
        
        if selected(receive_part1):
            v[0] = x
            v[1] = yield env.execute(c_in[1].receive())
            
        if selected(receive_part2):
            v[1] = x
            v[0] = yield env.execute(c_in[0].receive())
              
        yield env.execute(c_out.send(v))

We make a slight alteration to the exit model to show that the parts have indeed been assembled.

In [ ]:
@process
def Exit(env, c_in, N):
    for i in range(N):
        x = yield env.execute(c_in.receive())
        print(f"The Exit process received an item at t = {env.now:.1f} consisting of {len(x)} parts.")

Now we can run this model with `ta=3`, `ts=1`, and `N=10`.

In [ ]:
# Run the model
Assemble2PartsSystemModel(3, 1, 10)

### 10.3.1

A generalized assembly work station for `n` components is depicted in Figure 10.2.

| Figure 10.7:  Assembly for `n` components, with `m=n-1`. |
|-|
<img src="figures/10-7.png" width=500>
<a id='fig:10-7'></a>

Again, we first create the systen model. We substitute the buffers and servers with workstation submodel `W`, which is preceded by `n` generators. The workstation is succeeded by a single exit. The model is as follows:

In [ ]:
def AssembleMPartsSystemModel(ta, ts, N, n):
    env = Environment()
    a = [Channel(env) for j in range(n)]
    c = [Channel(env) for j in range(n)]
    b = Channel(env)
    Gs = [Generator(env, a[j], ta) for j in range(n)]
    W = WorkstationAssembly(env, a, b)
    E = Exit(env, b, N)
    env.run(until=E)

The entire work station submodel (the combined buffer processes and the assembly server process) is shown below.

The size of the list of channels `c_in` is determined during initialization of the workstation.
This size is used for the generation of the process buffers, and the accompanying channels. 

In [ ]:
def WorkstationAssembly(env, c_in, c_out):
    n = len(c_in)
    c_toS = [Channel(env) for j in range(n)]
    Bs = [Buffer(env, c_in[i], c_toS[i]) for i in range(n)]
    Ss = ServerAssemblyMParts(env, c_toS, c_out)

The assembly server process works in the same way as before, except for a generic `n` components, it is impossible to write a select statement explicitly. Instead, the list of communication events `receive_parts` is defined using *list comprehension*, which combined with `env.select(*receive_parts)` is used to unfold the alternatives.

The received components are again in `v`. Item `v[i]` is received from channel `c[i]`. The indices of the channels that have not provided an item are in the list `rec`. Initially, it contains all channels `0`$\ldots$`n`, that is, `rec = list(range(n))`. While `rec` still has a channel index to monitor, the `[c_in[i].receive() if (i in rec) else None for i in range(n)]` contains all possible communication events that are required to finalize the assembly process. For example, if `rec` contains `[0, 1, 4]` for `n=5`, then `receive_parts =[c_in[i].receive() if (i in rec) else None for i in range(n)]` is equivalent to:

    receive_parts = [c_in[0].receive(), c_in[1].receive(), None, None, c_in[4].receive()]
                     .
Which means that `yield env.select(*receive_parts)` will try to receive an item over either channels `c_in[0]`, `c_in[1]` or `c_in[4]`. 

After receiving an item, the index of the channel is removed from `rec` to prevent receiving a second item from the same channel. When all items have been received, the assembled component is sent away with `yield env.execute(c_out.send(v))`.

In [ ]:
@process
def ServerAssemblyMParts(env, c_in, c_out):
    n = len(c_in)
    v = [None]*n
    while True:
        rec = list(range(n))
        while len(rec)>0:
            receive_parts = [c_in[i].receive() if (i in rec) else None for i in range(n)]
            x = yield env.select(*receive_parts)
            for j in range(n):
                if selected(receive_parts[j]):
                    v[j] = x
                    rec.remove(j)
        yield env.execute(c_out.send(v))

The exit model remains the same as in the previous assembly system model.

In practical situations these assembly processes are performed in a more cascading manner:
two or three components are 'glued' together in one assemble process, followed in the next process by another assembly process.

In [ ]:
# Run the model
AssembleMPartsSystemModel(3, 1, 10, 4)

## 10.4 Exercises

### 10.4.1
Predict the resulting throughput and flow time for a deterministic case like in Section 10.1.1, with `ta = 4` and
    `ts = 5`. Verify the prediction with an experiment, and explain the result.

### 10.4.2 
Extend the model of Exercise 1 in Section 9.5 with a single deterministic server taking `4.0` time units to model the production capacity of the factory. Increase the number of products inserted by the generator, and measure the average flow time for
   1. A FIFO buffer with control policy `low = 0` and `high = 1`.
   2. A FIFO buffer with control policy `low = 1` and `high = 4`.
   3. A LIFO buffer with control policy `low = 1` and `high = 4`.

In [ ]:
@dataclass
class Product:
    id: int
    entrytime: float
                
@process
def Generator(env, c_out, c_signal, N):
    for i in range(N):
        yield env.execute(c_signal.receive())
        x = Product(id = i, entrytime = env.now)
        yield env.execute(c_out.send(x))
        
@process
def Exit(env, c_in, c_signal):
    mean_flowtime = 0.0
    i = 1
    while True:
        yield env.execute(c_signal.receive())
        x = yield env.execute(c_in.receive())
        flowtime = env.now - x.entrytime
        mean_flowtime = (i - 1) / i * mean_flowtime  +  flowtime / i
        print(f"The Exit process received {x.id}, with flowtime {flowtime:.2f}. The mean flowtime is {mean_flowtime:.2f}.")
        i = i+1
        
@process
def Controller(env, c_signal_gen, c_signal_exit, low, high):
    count = 0
    while True:
        while count < high:
            yield env.execute(c_signal_gen.send())
            count = count+1
        while count > low:
            yield env.execute(c_signal_exit.send()) 
            count = count-1
    
def model(low, high, N):
    env = Environment()
    sg = Channel(env)
    se = Channel(env)
    gf = Channel(env)
    fe = Channel(env)
    G = Generator(env, gf, sg, N)
    F = Factory(env, gf, fe)
    E = Exit(env, fe, se)
    C = Controller(env, sg, se, low, high)
    env.run()

---
**A.**

In [ ]:
# The factory is a submodel which contains both a FIFO buffer and a server
def Factory(env, c_in, c_out):
    ...

In [ ]:
model(low=0, high=1, N=..)

---
**B.**

In [ ]:
# 2b
model(low=1, high=4, N=..)

---
**C.**

In [ ]:
# 2c: The factory is a submodel which contains both a LIFO buffer and a server
def Factory(env, c_in, c_out):
    ...

In [ ]:
# 2c
model(low=1, high=4, N=..)

## 10.5 Answers to exercises

### Answer to 10.4.1

<details>
    <summary>[Click for the answer to 10.4.1]</summary>
   
In this model the generator will try to send items every 4 time units. However, the server needs 5 time units to process an item, and there is no buffer between the generator and server, so the actual interarrivaltime will be 5 time units. The predicted throughput is 1/5. The flow time of all items is 5 time units. Simulation of this model confirms these predictions.

</details>




### Answer to 10.4.2


<details>
    <summary>[Click for the answer to 10.4.2]</summary>
    
**A.** The factory can be modelled as following. The resulting mean flowtime is 8. The model is:

```python
def Factory(env, c_in, c_out):
    c_BtoS = Channel(env)
    B = Buffer(env, c_in, c_BtoS)
    S = Server(env, c_BtoS, c_out)

@process
def Buffer(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]

@process
def Server(env, c_in, c_out):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(4.0)
        yield env.execute(c_out.send(x))   
```

The model is ran with:
```python
model(low=0, high=1, N=10000)
```
---
**B.** The resulting mean flowtime becomes 16. The model is ran with:
```python
model(1, 4, 10000)
```
---
**C.** The factory is modelled the following way. The resulting mean flowtime is 12.
```python
def Factory(env, c_in, c_out):
    c_BtoS = Channel(env)
    B = Buffer(env, c_in, c_BtoS)
    S = Server(env, c_BtoS, c_out)

@process
def Buffer(env, c_in, c_out):
    xs = []
    while True:
        sending = c_out.send(xs[0]) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = [x] + xs 
        if selected(sending):
            xs = xs[1:]

@process
def Server(env, c_in, c_out):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(4.0)
        yield env.execute(c_out.send(x))
```
The model is ran with:
```python
    model(1, 4, 10000)
```

</details>